In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Installing the dependencies
import torch
!pip install ultralytics==8.0.20
!pip install roboflow --quiet
import os
os.getcwd()
HOME = os.getcwd()
%cd {HOME}
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
%pip install -r requirements.txt
from IPython import display
display.clear_output()
import ultralytics
ultralytics.checks()
from ultralytics import YOLO
import utils
display = utils.notebook_init()
from IPython.display import display, Image
from roboflow import Roboflow
from scipy.spatial import distance as dist
import numpy as np
import argparse
import cv2
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow


YOLOv5 🚀 v7.0-145-g94714fe Python-3.9.16 torch-2.0.0+cu118 CPU


Setup complete ✅ (2 CPUs, 12.7 GB RAM, 23.2/107.7 GB disk)


#Yolov5 Training and Model Saving

In [ ]:
#Insstalling the Dataset key from roboflow#
%cd /content/yolov5
!pip install -q roboflow
from roboflow import Roboflow
rf = Roboflow(api_key="NADQgoMx6u7n2Mg3BQWx")
project = rf.workspace("crystaltracking").project("ipak4tracking")
dataset = project.version(1).download("yolov5")

[Errno 2] No such file or directory: '/content/yolov5'
/content
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 KB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 KB 8.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 KB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 KB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 KB 8.5 MB/s eta 0:00:00
loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to iPak4Tracking-1 in yolov5pytorch:: 100%|██████████| 4664/4664 [00:00<00:00, 9725.05it/s] 


In [ ]:
!python train.py --img 640 --batch 32 --epochs 100 --data {dataset.location}/data.yaml --weights yolov5m.pt 

#Image Resizing

In [ ]:
path = '/content/drive/MyDrive/ProteinTracking/iPAK4Tracking/24'
for id in os.listdir(path):
  print(id)

In [ ]:
#Rewrite the figure into size of 640x640#
path = '/content/drive/MyDrive/ProteinTracking/iPAK4Tracking/25'
for id in os.listdir(path):
  video_frame = cv2.imread('%s/%s'%(path,id))
  print(id[:-4],video_frame.shape)
  video_frame = cv2.resize(video_frame,(640,640),interpolation=cv2.INTER_AREA)
  cv2.imwrite('/content/drive/MyDrive/ProteinTracking/iPAK4Tracking/ExtraRawImage/25_%s.jpg'%(id[:-4]),video_frame)
  

In [ ]:
#Generate the pseudo-label by v5x#
import torch
model = torch.hub.load('/content/yolov5', 'custom',path='/content/drive/MyDrive/ProteinTracking/iPAK4Tracking/v5x_100ep/best_v5x_32batch.pt',source='local',force_reload=True)
path = '/content/drive/MyDrive/ProteinTracking/iPAK4Tracking/ExtraRawImage'
for id in os.listdir(path):

  result = model('%s/%s'%(path,id))
  print(id)

  # Open the file in write mode
  file = open("/content/drive/MyDrive/ProteinTracking/iPAK4Tracking/ExtraRawLabel/%s.txt"%(id[:-4]), "w")
  result.pred[0][:,:4].cpu().numpy()
  # Write label to the file
  for i in range(result.pred[0].shape[0]):
    xyxy = result.pred[0].cpu().numpy()[i,:4]
    x_cen = (xyxy[2]+xyxy[0])/(2*640)
    y_cen = (xyxy[3]+xyxy[1])/(2*640)
    x_wid = (xyxy[2]-xyxy[0])/640
    y_wid = (xyxy[3]-xyxy[1])/640
    clss = int(result.pred[0].cpu().numpy()[i,5])
    if i < result.pred[0].shape[0]-1:
      file.write("%s %s %s %s %s\n"%(clss, x_cen, y_cen, x_wid, y_wid))
    else:
      file.write("%s %s %s %s %s"%(clss,x_cen, y_cen, x_wid, y_wid))
  # Close the file
  file.close()

In [ ]:
print(len(os.listdir('/content/drive/MyDrive/ProteinTracking/iPAK4Tracking/ExtraRawLabel')))

1232


#ByteTrack and Radon Transform

In [ ]:
%cd {HOME}
!git clone https://github.com/ifzhang/ByteTrack.git
!pip install loguru
!pip install scikit-image
!pip install tqdm
!pip install onnx==1.8.1
!pip install onnxruntime==1.8.0
!pip install onnx-simplifier==0.3.5
!pip install Pillow
!pip install thop
!pip install ninja
!pip install tabulate
!pip install tensorboard
!pip install lap
!pip install motmetrics
!pip install filterpy
!pip install
!cd ByteTrack && pip3 install -q -r requirements.txt
!cd ByteTrack && python3 setup.py -q develop
!pip install -q cython_bbox
!pip install -q onemetric

from IPython import display
display.clear_output()


import sys
sys.path.append(f"{HOME}/ByteTrack")


import yolox
print("yolox.__version__:", yolox.__version__)
from yolox.tracker.byte_tracker import BYTETracker, STrack
from onemetric.cv.utils.iou import box_iou_batch
from dataclasses import dataclass

@dataclass(frozen=True)
class BYTETrackerArgs:
    track_thresh: float = 0.25 #default 0.25
    track_buffer: int = 35#default 30
    match_thresh: float = 0.90 #default 0.8
    aspect_ratio_thresh: float = 3.0 #default 3.0
    min_box_area: float = 1.0 #default 1.0
    mot20: bool = False
!pip install supervision==0.1.0
import supervision
print("supervision.__version__:", supervision.__version__)
from supervision.draw.color import ColorPalette
from supervision.geometry.dataclasses import Point
from supervision.video.dataclasses import VideoInfo
from supervision.video.source import get_video_frames_generator
from supervision.video.sink import VideoSink
from supervision.notebook.utils import show_frame_in_notebook
from supervision.tools.detections import Detections, BoxAnnotator
from typing import List
# converts Detections into format that can be consumed by match_detections_with_tracks function
def detections2boxes(detections: Detections) -> np.ndarray:
    return np.hstack((
        detections.xyxy,
        detections.confidence[:, np.newaxis]
    ))

# converts List[STrack] into format that can be consumed by match_detections_with_tracks function
def tracks2boxes(tracks: List[STrack]) -> np.ndarray:
    return np.array([
        track.tlbr
        for track
        in tracks
    ], dtype=float)

# matches our bounding boxes with predictions
def match_detections_with_tracks(
    detections: Detections, 
    tracks: List[STrack]
) -> Detections:
    if not np.any(detections.xyxy) or len(tracks) == 0:
        return np.empty((0,))

    tracks_boxes = tracks2boxes(tracks=tracks)
    iou = box_iou_batch(tracks_boxes, detections.xyxy)
    track2detection = np.argmax(iou, axis=1)
    
    tracker_ids = [None] * len(detections)
    
    for tracker_index, detection_index in enumerate(track2detection):
        if iou[tracker_index, detection_index] != 0:
            tracker_ids[detection_index] = tracks[tracker_index].track_id

    return tracker_ids

yolox.__version__: 0.1.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
supervision.__version__: 0.1.0


In [ ]:
import pandas as pd
model = torch.hub.load('/content/yolov5', 'custom',path='/content/drive/MyDrive/ProteinTracking/iPAK4Tracking/v5n_100ep/best_v5n_32batch.pt',source='local',force_reload=True)
result = model('/content/drive/MyDrive/ProteinTracking/iPAK4Tracking/ExtraRawImage/21_sec11_0049.jpg').pred[0].cpu().cpu().numpy()
result[:,5]

YOLOv5 🚀 v7.0-128-gb96f35c Python-3.9.16 torch-1.13.1+cu116 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)

Fusing layers... 
Model summary: 157 layers, 1760518 parameters, 0 gradients, 4.1 GFLOPs
Adding AutoShape... 


array([          0,           0,           0,           0,           0], dtype=float32)

In [ ]:
#Extracing BBOX from each id in each frame Version#
import numpy as np
import matplotlib.pyplot as plt
import cv2
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow
from skimage.data import shepp_logan_phantom
from skimage.transform import radon, rescale
import time
working_path = '/content/drive/MyDrive/ProteinTracking/iPAK4Tracking/v5np_100ep' #Folder of your current model
if not os.path.exists('%s/%s'%(working_path,'ResultVideos')):
  os.mkdir('%s/%s'%(working_path,'ResultVideos'))
#Predicting the video content#
def video_generator(path,model,videoid,target_info):
  # dict maping class_id to class_name
  CLASS_NAMES_DICT = {0: 'iPak4'}
  # class_ids of interest - can be checked from the model class
  CLASS_ID = [0] # Here 0 is ipak4
  # create frame generator
  SOURCE_VIDEO_PATH = path+'/InputVideo/'+videoid

  # set the output
  TARGET_VIDEO_PATH = '%s/ResultVideos/%s_result.mp4'%(path,videoid[:-4])
  VideoInfo.from_video_path(SOURCE_VIDEO_PATH)

  from tqdm.notebook import tqdm
  # create BYTETracker instance
  byte_tracker = BYTETracker(BYTETrackerArgs())
  # create VideoInfo instance
  video_info = VideoInfo.from_video_path(SOURCE_VIDEO_PATH)
  # create frame generator
  generator = get_video_frames_generator(SOURCE_VIDEO_PATH)
  # create instance of BoxAnnotator
  box_annotator = BoxAnnotator(color=ColorPalette(), thickness=1, text_thickness=1, text_scale=1)
  # create a list to store the time consumption of object tracking for each frame
  Time_consumption = []
  
  # open target video file
  with VideoSink(TARGET_VIDEO_PATH, video_info) as sink:
      Fiber_info = {} # Initialize a new dictionary to record the length change in each frame for each fiber

      # loop over video frames
      frame_n = 0
      for frame in tqdm(generator, total=video_info.total_frames):
          start_time = time.time()
          # model prediction on single frame and conversion to supervision Detections
          results = model(frame).pred[0].cpu().cpu().numpy()
          detections = Detections(
              xyxy=results[:, :4],
              confidence=results[:, 4],
              class_id=results[:, 5].astype(int),
          )
          
          # filtering out detections with unwanted classes
          mask = np.array([class_id in CLASS_ID for class_id in detections.class_id], dtype=bool)
          detections.filter(mask=mask, inplace=True)
          # tracking detections
          tracks = byte_tracker.update(
              output_results=detections2boxes(detections=detections),
              img_info=frame.shape,
              img_size=frame.shape
          )
          tracker_id = match_detections_with_tracks(detections=detections, tracks=tracks)
          detections.tracker_id = np.array(tracker_id)
          # filtering out detections without trackers
          mask = np.array([tracker_id is not None for tracker_id in detections.tracker_id], dtype=bool)
          detections.filter(mask=mask, inplace=True)
          # format custom labels
          
          labels = [
              f"#{tracker_id} {CLASS_NAMES_DICT[class_id]} {confidence:0.2f}"
              for _, confidence, class_id, tracker_id
              in detections
            
          ]
          
          # Recording the information of target in this frame
          
          #print(detections.xyxy.shape) #The shape is (1,n), so two index should be input to locate the coordinates
          # Iterating each target id
          for idx_fiber in range(len(detections.tracker_id)):
            # In case the target occurs for the first time
            if detections.tracker_id[idx_fiber] not in Fiber_info.keys():
              Fiber_info[detections.tracker_id[idx_fiber]] = []
            info = np.append(detections.xyxy[idx_fiber],frame_n)
            Fiber_info[detections.tracker_id[idx_fiber]].append(info) #{fiber_id:[[xmin ymin xmax ymax frame_id]]}
            


            # Sice the Image bounded by BBOX
            
          frame_n +=1
          # annotate and display frame
          frame = box_annotator.annotate(frame=frame, detections=detections, labels=labels)
          end_time = time.time()
          processing_speed = end_time -start_time
          Time_consumption.append(processing_speed)
          sink.write_frame(frame)
  Time_consumption = np.asarray(Time_consumption)
  print('Time for processing %s is %s'%(videoid,np.mean(Time_consumption)))
  target_info = Fiber_info  
  return target_info
  #videoid = os.listdir('/content/drive/MyDrive/ProteinTracking/iPAK4Tracking/InputVideo')



#Generating the tracking files 
videoid = os.listdir(working_path+'/InputVideo')
Fiber_info ={}

#Load-in the pretrained YOLO model
MODEL_weight = '%s/best.pt'%(working_path)
model = torch.hub.load('/content/yolov5', 'custom',path=MODEL_weight,source='local',force_reload=True)

for vid in videoid:
  
  Fiber_info = video_generator(working_path,model,vid,Fiber_info)
  
  #Save the data points into numpy file
  if not os.path.exists('%s/%s'%(working_path,vid[:-4])):
    os.mkdir('%s/%s'%(working_path,vid[:-4]))

  np.save('%s/%s/%s.npy'%(working_path,vid[:-4],vid[:-4]),Fiber_info)


YOLOv5 🚀 v7.0-133-gcca5e21 Python-3.9.16 torch-1.13.1+cu116 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)



[Errno 2] No such file or directory: '/usr/local/lib/python3.9/dist-packages/pyparsing-3.0.9.dist-info/METADATA'


Fusing layers... 
Model summary: 157 layers, 1760518 parameters, 0 gradients, 4.1 GFLOPs
Adding AutoShape... 


  0%|          | 0/264 [00:00<?, ?it/s]

Time for processing FOV35_section5.mp4 is 0.016850254752419212


...

Time for processing FOV35_section14.mp4 is 0.01710445230657404


  0%|          | 0/264 [00:00<?, ?it/s]

Time for processing FOV35_section7.mp4 is 0.016144156455993652


  0%|          | 0/264 [00:00<?, ?it/s]

Time for processing FOV35_section8.mp4 is 0.017801230603998356


In [ ]:
print(len(os.listdir('/content/drive/MyDrive/ProteinTracking/iPAK4Tracking/v5np_100ep/InputVideo')))

62


In [ ]:
#Installing dependencies#
import os
import cv2 as cv
from PIL import Image
import numpy as np
from matplotlib.figure import figaspect
from skimage.transform import radon, rescale
import csv
working_path = '/content/drive/MyDrive/ProteinTracking/iPAK4Tracking/v5np_100ep' #Folder of your current model
if not os.path.exists('%s/%s'%(working_path,'ResultCSV')):
  os.mkdir('%s/%s'%(working_path,'ResultCSV'))

def generate_result(videoid,crystal_id):
  # open target video file
  from tqdm.notebook import tqdm
  # create VideoInfo instance
  SOURCE_VIDEO_PATH = working_path+'/InputVideo/'+videoid+'.mp4'
  video_info = VideoInfo.from_video_path(SOURCE_VIDEO_PATH)
  # create frame generator
  generator = get_video_frames_generator(SOURCE_VIDEO_PATH)
  # define the saved video location
  Saved_path = working_path+'/'+videoid+'/Crystal%s_radon_result.mp4'%crystal_id
  CSV_path = working_path+'/ResultCSV/%s_Crystal%s'%(videoid,crystal_id)
  # load in the pre-saved tracking inforamtion
  tracking_info = np.load('%s/%s/%s.npy'%(working_path,videoid,videoid), allow_pickle=True).item()
  frame_id = 0
  distances=[]
  video = cv.VideoWriter(Saved_path,cv.VideoWriter_fourcc(*"mp4v"),20,(500,500)) #(int(target[1].split('x')[0]),int(target[1].split('x')[1]))
  plt.ion()

  with open("%s.csv"%(CSV_path),"w") as csvfile: 
    writer = csv.writer(csvfile)
    #Define the column name
    writer.writerow(["Frame","Distance(In Pixel)"])
    for frame in tqdm(generator, total=video_info.total_frames):
      #print(frame_id)
      if len(tracking_info[crystal_id])>0 and frame_id == int(tracking_info[crystal_id][0][-1]):
        #print(tracking_info[crystal_id][0])
        image = frame[int(tracking_info[crystal_id][0][1]):int(tracking_info[crystal_id][0][3])+1,int(tracking_info[crystal_id][0][0]):int(tracking_info[crystal_id][0][2])+1,:]
        image_sqz = np.empty((image.shape[0],image.shape[1]))
        image_sqz[:,:] = image[:,:,0] 
        
        #Radon Transform#
        with plt.ioff():
          fig, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4, figsize=(10, 10))
          ax1.set_title("Original")
          ax1.imshow(image_sqz, cmap=plt.cm.Greys_r)

          #Performing Radon Transform#
          theta = np.linspace(0.,180., max(image_sqz.shape), endpoint=False)
          sinogram = radon(image_sqz, theta=theta)
          #print(sinogram)
          #Finding the maximum value from forward radon transform
          s_ori,a_ori = np.where(sinogram==np.max(sinogram)) #obatin the index of the maximum value in sinogram
          #print(s_ori,a_ori)
          s_ori=s_ori[0];a_ori=a_ori[0]
          #Interpretting the value in real physical meaning
          s = s_ori-sinogram.shape[0]/2 #mapping the index back to the original domain
          a_ori = a_ori*180/sinogram.shape[1] #mapping the index back to the origin
          a = a_ori/180 *np.pi #converting the degree into radian
          #print(s_ori,a_ori,s,a)
          ax2.set_title("Radon transform\n(Sinogram)")
          ax2.set_xlabel("Projection angle (deg)")
          ax2.set_ylabel("Projection position (pixels)")
          ax2.imshow(sinogram, extent=(0, 180, -sinogram.shape[0]/2.0, sinogram.shape[0]/2.0),cmap=plt.cm.Greys_r, aspect='auto')

          #Calculating the function of integration line
          s=-s
          x_intersect = image_sqz.shape[0]//2 + s*np.sin(a); y_intersect = image_sqz.shape[1]//2 - s*np.cos(a)
          b = y_intersect - np.tan(a)*x_intersect

          #Calculating the corresponding pixels on the integration line from x- and y-axes respectively
          x_real = np.asarray([i for i in range(image_sqz.shape[0])])
          y_real = np.asarray([i for i in range(image_sqz.shape[1])])

          x_cal_ceil = np.floor((y_real-b)/(np.tan(a)))
          #x_cal_floor = np.floor((y_real-b)/(np.tan(a)))

          y_cal_ceil = np.ceil(x_real*np.tan(a)+b)
          #y_cal_floor = np.floor(x_real*np.tan(a)+b)

          #filtering the x and y to ensure they lie within the image
          x_filter =[]; y_filter=[]
          y_draw = []; x_draw =[]

          for i in range(len(y_cal_ceil)):
            if y_cal_ceil[i] > 0 and y_cal_ceil[i]<image_sqz.shape[1]:
              y_filter.append(y_cal_ceil[i])
              x_draw.append(x_real[i])

          for i in range(len(x_cal_ceil)):
            if x_cal_ceil[i] > 0 and x_cal_ceil[i]<image_sqz.shape[0]:
              x_filter.append(x_cal_ceil[i])
              y_draw.append(y_real[i])

          

          if len(x_filter)>0 and len(y_filter)>0:

              x_sort = x_filter+x_draw
              y_sort = y_filter+y_draw
              min_pt = np.array([min(x_sort),min(y_sort)])
              max_pt = np.array([max(x_sort),max(y_sort)])
              #min_pt=np.array([x_filter[0],y_filter[0]])
              #max_pt=np.array([x_filter[-1],y_filter[-1]])
              distance = np.linalg.norm(min_pt-max_pt)
              #print(min_pt,max_pt,distance)
              distances.append(distance)
              writer.writerow([frame_id,distance])

          ax3.set_title("Fiber mask from Radon Transform")
          ax3.imshow(image_sqz, cmap=plt.cm.Greys_r)
          ax3.plot(y_filter,x_draw,'gs')
          ax3.plot(y_draw,x_filter,'gs')
          ax3.plot(image_sqz.shape[1]//2, image_sqz.shape[0]//2, 'o')

          ax4.set_title("Fiber Length")
          ax4.set_aspect('equal')
          ax4.plot(np.linspace(0,len(distances),len(distances)),distances)
          #print(np.linspace(0,len(distances),len(distances)))
          #print(distances)
          fig.tight_layout()          
          plt.savefig('temp.png')
          plt.close()

          video_frame = cv2.imread('temp.png')
          #print('temp%s.png'%frame_id)
          video_frame = cv2.resize(video_frame,(500,500),interpolation=cv2.INTER_AREA)
          video.write(video_frame)
          tracking_info[crystal_id].pop(0)
          #Write in the columns_name
          writer.writerow([frame_id,distance])
      frame_id += 1

  video.release()




In [ ]:
print(len(os.listdir('/content/drive/MyDrive/ProteinTracking/iPAK4Tracking/v5np_100ep/ResultCSV')))

FileNotFoundError: ignored

In [ ]:
generate_result('FOV33_section9',3194)#3637 3618

  0%|          | 0/264 [00:00<?, ?it/s]

divide by zero encountered in true_divide
invalid value encountered in true_divide


#Advanced Radon Transform Processing

In [ ]:
#Installing dependencies#
import os
import cv2 as cv
from PIL import Image
import numpy as np
from matplotlib.figure import figaspect
from skimage.transform import radon, rescale
import csv
import math
import imutils

working_path = '/content/drive/MyDrive/CV/ProteinTracking/iPAK4Tracking/v5np_100ep' #Folder of your current model
if not os.path.exists('%s/%s'%(working_path,'ResultCSV')):
  os.mkdir('%s/%s'%(working_path,'ResultCSV'))




def generate_result(videoid,crystal_id):
  # open target video file
  from tqdm.notebook import tqdm
  # create VideoInfo instance
  SOURCE_VIDEO_PATH = working_path+'/InputVideo/'+videoid+'.mp4'
  video_info = VideoInfo.from_video_path(SOURCE_VIDEO_PATH)
  # create frame generator
  generator = get_video_frames_generator(SOURCE_VIDEO_PATH)
  # define the saved video location
  Saved_path = working_path+'/'+videoid+'/Crystal%s_radon_result.mp4'%crystal_id
  CSV_path = working_path+'/ResultCSV/%s_Crystal%s'%(videoid,crystal_id)
  # load in the pre-saved tracking inforamtion
  tracking_info = np.load('%s/%s/%s.npy'%(working_path,videoid,videoid), allow_pickle=True).item()
  frame_id = 0
  distances=[]
  video = cv.VideoWriter(Saved_path,cv.VideoWriter_fourcc(*"mp4v"),20,(500,500)) #(int(target[1].split('x')[0]),int(target[1].split('x')[1]))
  plt.ion()

  with open("%s.csv"%(CSV_path),"w") as csvfile: 
    writer = csv.writer(csvfile)
    #Define the column name
    writer.writerow(["Frame","Distance(In Pixel)"])
    for frame in tqdm(generator, total=video_info.total_frames):

      if len(tracking_info[crystal_id])>0 and frame_id == int(tracking_info[crystal_id][0][-1]): #{fiber_id:[[xmin ymin xmax ymax frame_id]]}
        #print(tracking_info[crystal_id][0])

        #Crop the target region larger than the bbox in case the bbox didn't cover the whole ipak4
        xmin = math.floor(int(tracking_info[crystal_id][0][0]) * 0.975)
        xmax = math.ceil(int(tracking_info[crystal_id][0][2])*1.025)
        ymin = math.floor(int(tracking_info[crystal_id][0][1]) * 0.975);
        ymax = math.ceil(int(tracking_info[crystal_id][0][3])*1.025)
        if xmax > frame.shape[0]:
          xmax = frame.shape[0]
        if ymax > frame.shape[1]:
          ymax = frame.shape[1]
        #image = frame[int(tracking_info[crystal_id][0][1]):int(tracking_info[crystal_id][0][3])+1,int(tracking_info[crystal_id][0][0]):int(tracking_info[crystal_id][0][2])+1,:]
        image = frame[ymin:ymax+1,xmin:xmax+1,:]
        image_sqz = np.empty((image.shape[0],image.shape[1]))
        image_sqz[:,:] = image[:,:,0]
        #Performing Radon Transform#
        theta = np.linspace(0.,180., max(image_sqz.shape), endpoint=False)
        sinogram = radon(image_sqz, theta=theta)
        #print(sinogram)
        #Finding the maximum value from forward radon transform
        s_ori,a_ori = np.where(sinogram==np.max(sinogram)) #obatin the index of the maximum value in sinogram
        #print(s_ori,a_ori)
        s_ori=s_ori[0];a_ori=a_ori[0]
        #Interpretting the value in real physical meaning
        s = s_ori-sinogram.shape[0]/2 #mapping the index back to the original domain
        a_ori = a_ori*180/sinogram.shape[1] #mapping the index back to the origin
        a = a_ori/180 *np.pi #converting the degree into radian
        #print(s_ori,a_ori,s,a)

        #Rotate the image to horizontal
        img_ro = imutils.rotate_bound(image_sqz, -(90-a_ori))
        intensity = img_ro[int(img_ro.shape[0]//2-s)-5:int(img_ro.shape[0]//2-s)+6,:]
        intensity = np.mean(intensity,axis=0)
        intensity = intensity[intensity!=0]
        ix = np.asarray(np.where(intensity>=(np.mean(intensity)-0.8*np.std(intensity))))[0]

        #Radon Transform#
        if ix.shape[0]>0:
          left = ix[0]
          right = ix[-1]
          with plt.ioff():
            fig, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4, figsize=(10, 10))
            ax1.set_title("Region of fiber")
            ax1.imshow(image_sqz, cmap=plt.cm.Greys_r)


            ax2.set_title("Radon transform\n(Sinogram)")
            ax2.set_xlabel("Projection angle (deg)")
            ax2.set_ylabel("Projection position (pixels)")
            ax2.imshow(sinogram, extent=(0, 180, -sinogram.shape[0]/2.0, sinogram.shape[0]/2.0),cmap=plt.cm.Greys_r, aspect='auto')


            ax3.set_title("Mask from Radon Transform")
            ax3.imshow(img_ro, cmap=plt.cm.Greys_r)
            #ax3.plot(img_ro.shape[1]//2, img_ro.shape[0]//2, 'o') #indicating the center
            
            ax4.set_title("Fiber Length")
            ax4.set_aspect('equal')

            #ix = np.asarray(np.where(intensity>=(np.max(intensity)*0.3)))[0]


            distances.append(abs(right-left))
            ax3.plot(np.arange(left,right),np.asarray([img_ro.shape[0]//2-s]*(right-left)),color='red',linewidth=3) #indicating the position of maximum radon transform
            #ax4.plot(np.arange(intensity.shape[0]),intensity)
            #ax4.plot(left,intensity[left],'o')
            #ax4.plot(right,intensity[right],'o')
            ax4.plot(np.linspace(0,len(distances),len(distances)),distances)

            #ax4.imshow(imutils.rotate_bound(img_ro,a_ori-90),cmap=plt.cm.Greys_r)


            fig.tight_layout()          
            plt.savefig('temp.png')
            plt.close()

            
            video_frame = cv2.imread('temp.png')
            #print('temp%s.png'%frame_id)
            video_frame = cv2.resize(video_frame,(500,500),interpolation=cv2.INTER_LANCZOS4)
            video.write(video_frame)
            tracking_info[crystal_id].pop(0) #Popout the current frame to process the next one
            #Write in the columns_name
            writer.writerow([frame_id,abs(right-left)])
      frame_id += 1

  video.release()


In [ ]:
generate_result('FOV30_section6',2377)

  0%|          | 0/264 [00:00<?, ?it/s]

In [ ]:
print(len(os.listdir('/content/drive/MyDrive/CV/ProteinTracking/iPAK4Tracking/v5np_100ep/ResultCSV')))

39
